In [3]:
# Uncomment the lines below and run the installations
#!pip install torch
#!pip install torchvision
#!pip install opencv-python
#!pip install gradio

In [6]:
# Import libs
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import os

In [12]:
# Calculating the mean and standard deviation of the data set
import os
import numpy as np
import cv2

def load_images_from_folder(folder, target_size=(256, 256)):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, target_size)
            images.append(img)
    return images

def compute_mean_std(images):
    stacked_images = np.stack(images, axis=0)
    mean = np.mean(stacked_images)
    std = np.std(stacked_images)
    return mean, std

folder_path = 'C:\\Users\\Khomp\Desktop\\Projetos SBC 2025\\ResNet50\\Health-projects\\Chest_Xray-classification\\checandoMEDIAeSD'
images = load_images_from_folder(folder_path)
mean, std = compute_mean_std(images)

print(f'Mean: {mean}')
print(f'Standard Deviation: {std}')


Mean: 112.30059990516075
Standard Deviation: 85.98406628052774


In [13]:
# Defining data transformations for grayscale images with the computed mean and std
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([112.3006 / 255], [85.9841 / 255])  # Normalize using the computed mean and std
    ]),
    'val': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([112.3006 / 255], [85.9841 / 255])  # Normalize using the computed mean and std
    ]),
}

In [15]:
# Defining the data directory
data_dir = 'dataset'

# Creating data loaders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
#image_datasets

In [16]:
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['train'].classes
class_names

{'train': 91, 'val': 13}


['healthy', 'tuberculosis']

In [20]:
# Load the pre-trained ResNet-50 model
model = models.resnet50(pretrained=True)

In [21]:
# Freeze all layers except the final classification layer
for name, param in model.named_parameters():
    if "fc" in name:  # Unfreeze the final classification layer
        param.requires_grad = True
    else:
        param.requires_grad = False

In [22]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)  # Use all parameters

In [23]:
# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [24]:
# Training loop
num_epochs = 2
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Convert grayscale images to RGB
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1, 3, 1, 1)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]

        print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training complete!")

train Loss: 2.9721 Acc: 0.4176
val Loss: 1.4242 Acc: 0.4615
train Loss: 0.8150 Acc: 0.5604
val Loss: 1.0390 Acc: 0.5385
Training complete!


In [25]:
# =========================== Evaluation =========================

from sklearn.metrics import precision_score, recall_score, accuracy_score
import numpy as np
import torch

def evaluate_model(model, dataloaders, dataset_sizes, device):
    model.eval()

    for phase in ['train', 'val']:
        all_preds = []
        all_labels = []

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Converting grayscale images to RGB
            if inputs.shape[1] == 1:
                inputs = inputs.repeat(1, 3, 1, 1)

            with torch.set_grad_enabled(False):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # Converting lists to NumPy arrays
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)

        accuracy = accuracy_score(all_labels, all_preds)
        precision = precision_score(all_labels, all_preds, average='weighted')
        recall = recall_score(all_labels, all_preds, average='weighted')

        print(f'{phase} Accuracy: {accuracy:.4f}')
        print(f'{phase} Precision: {precision:.4f}')
        print(f'{phase} Recall: {recall:.4f} \n')
        

# Calling the function after training the model
evaluate_model(model, dataloaders, dataset_sizes, device)

train Accuracy: 0.5604
train Precision: 0.5555
train Recall: 0.5604 

val Accuracy: 0.5385
val Precision: 0.3590
val Recall: 0.5385 

